In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

2023-02-21 11:34:49.195589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 11:34:49.305685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 11:34:49.305701: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 11:34:49.826120: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df1 = pd.read_csv("UNSW_NB15_testing-set.csv")
df2 = pd.read_csv("UNSW_NB15_training-set.csv")
df = pd.concat([df1,df2])
col2encode = ["proto","service","state"]

proto_name = df["proto"].unique()
proto_count = df["proto"].value_counts()
d_proto = {}
for p in proto_name:
    d_proto.update({p:proto_count[p]})
print(d_proto)
n_value_to_extract = 20
d_proto_20 = {}
for i in range(n_value_to_extract):
    d_proto_20.update({max(d_proto):d_proto[max(d_proto)]})
    d_proto.pop(max(d_proto))
print(d_proto_20)


ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df = df.drop(columns=["attack_cat","id"])
df_ohe = pd.concat([df,ohe_col],axis=1)
d_attack = {0:False,1:True}
label = df_ohe["label"].map(d_attack)
df_ohe = df_ohe.drop(columns=["label"])

print(df1.shape)
print(df2.shape)
print(df_ohe.shape)

X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)

NCOMPONENTS = 36

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)

print(X_pca_train.shape)
print(X_pca_test.shape)

X_pca_train  = X_pca_train.reshape(X_pca_train.shape[0],6,6)
X_pca_test  = X_pca_test.reshape(X_pca_test.shape[0],6,6)

print(X_pca_train.shape)
print(X_pca_test.shape)


{'tcp': 123041, 'udp': 92701, 'arp': 3846, 'ospf': 3271, 'icmp': 15, 'igmp': 48, 'rtp': 1, 'ddp': 132, 'ipv6-frag': 132, 'cftp': 132, 'wsn': 132, 'pvp': 132, 'wb-expak': 132, 'mtp': 132, 'pri-enc': 132, 'sat-mon': 132, 'cphb': 132, 'sun-nd': 255, 'iso-ip': 132, 'xtp': 132, 'il': 132, 'unas': 15599, 'mfe-nsp': 132, '3pc': 132, 'ipv6-route': 132, 'idrp': 132, 'bna': 132, 'swipe': 253, 'kryptolan': 132, 'cpnx': 132, 'rsvp': 264, 'wb-mon': 132, 'vmtp': 132, 'ib': 132, 'dgp': 132, 'eigrp': 132, 'ax.25': 132, 'gmtp': 132, 'pnni': 132, 'sep': 251, 'pgm': 132, 'idpr-cmtp': 132, 'zero': 132, 'rvd': 132, 'mobile': 253, 'narp': 132, 'fc': 132, 'pipe': 132, 'ipcomp': 132, 'ipv6-no': 132, 'sat-expak': 132, 'ipv6-opts': 132, 'snp': 132, 'ipcv': 132, 'br-sat-mon': 132, 'ttp': 132, 'tcf': 132, 'nsfnet-igp': 132, 'sprite-rpc': 132, 'aes-sp3-d': 132, 'sccopmce': 132, 'sctp': 1474, 'qnx': 132, 'scps': 132, 'etherip': 132, 'aris': 132, 'pim': 253, 'compaq-peer': 132, 'vrrp': 132, 'iatp': 132, 'stp': 132, 

In [3]:

from keras import Input
from keras.regularizers import l1
from keras.layers import MaxPooling2D, Flatten, BatchNormalization, Dense
from qkeras import QConv2DBatchnorm, QActivation, QDense

x = x_in = Input(shape=(6,6,1))

x = QConv2DBatchnorm(32, kernel_size=(3,3), strides=(1,1),
                     kernel_quantizer="quantized_bits(6,0,alpha=1)",
                     bias_quantizer="quantized_bits(6,0,alpha=1)",
                     kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                     name='fused_convbn')(x)
x = QActivation('quantized_relu(6)',name='conv_act1')(x)
x = MaxPooling2D(pool_size = (2,2),name='pool')(x)
x = Flatten()(x)
x = BatchNormalization(name='bn_dense')(x)
x_out = Dense(1,
           name='output_dense')(x)
#x_out = Activation('softmax',name='output_softmax')(x)
model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

model.summary()

2023-02-21 11:34:56.807264: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 11:34:56.807313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-21 11:34:56.807340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-21 11:34:56.807365: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-21 11:34:56.807390: W tensorfl

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 6, 1)]         0         
                                                                 
 fused_convbn (QConv2DBatchn  (None, 4, 4, 32)         449       
 orm)                                                            
                                                                 
 conv_act1 (QActivation)     (None, 4, 4, 32)          0         
                                                                 
 pool (MaxPooling2D)         (None, 2, 2, 32)          0         
                                                                 
 flatten (Flatten)           (None, 128)               0 

In [4]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics =['accuracy']
)

model.fit(X_pca_train, y_train, batch_size=64, epochs=10,validation_data=(X_pca_test,y_test))

Epoch 1/10
3020/3020 [==============================] - 23s 7ms/step - loss: 1.0043 - accuracy: 0.8164 - val_loss: 0.6103 - val_accuracy: 0.8701
Epoch 2/10
3020/3020 [==============================] - 23s 7ms/step - loss: 0.4630 - accuracy: 0.8659 - val_loss: 0.2714 - val_accuracy: 0.8884
Epoch 3/10
3020/3020 [==============================] - 22s 7ms/step - loss: 0.4715 - accuracy: 0.8754 - val_loss: 0.3062 - val_accuracy: 0.8771
Epoch 4/10
3020/3020 [==============================] - 22s 7ms/step - loss: 0.3287 - accuracy: 0.8801 - val_loss: 0.2285 - val_accuracy: 0.8931
Epoch 5/10
3020/3020 [==============================] - 22s 7ms/step - loss: 0.8414 - accuracy: 0.8471 - val_loss: 1.8392 - val_accuracy: 0.7439
Epoch 6/10
3020/3020 [==============================] - 22s 7ms/step - loss: 2.0508 - accuracy: 0.7340 - val_loss: 2.8643 - val_accuracy: 0.6776
Epoch 7/10
3020/3020 [==============================] - 21s 7ms/step - loss: 1.7422 - accuracy: 0.7765 - val_loss: 0.5141 - val_ac

In [5]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude

qmodel  = strip_pruning(model)

In [6]:
import hls4ml
import plotting
import os

os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

# Then the QKeras model
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config_q = hls4ml.utils.config_from_keras_model(qmodel, granularity='name')
hls_config_q['Model']['ReuseFactor'] = 1
hls_config_q['Model']['Precision'] = 'ap_fixed<6,0>'
  
cfg_q = hls4ml.converters.create_config(backend='Vivado')
cfg_q['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg_q['HLSConfig']  = hls_config_q
cfg_q['KerasModel'] = qmodel
cfg_q['OutputDir']  = 'quantized_pruned_cnn/'
cfg_q['XilinxPart'] = 'xc7z020clg484-1'
cfg_q['ClockPeriod'] = 10
  
hls_model_q = hls4ml.converters.keras_to_hls(cfg_q)
hls_model_q.compile()


Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: fused_convbn, layer type: QConv2DBatchnorm
Layer name: conv_act1, layer type: QActivation
Layer name: pool, layer type: MaxPooling2D
Layer name: bn_dense, layer type: BatchNormalization
Layer name: output_dense, layer type: Dense
  -> Activation (linear), layer name: output_dense
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 6, 6, 1]], output shape: [None, 6, 6, 1]
Layer name: fused_convbn, layer type: QConv2DBatchnorm, input shapes: [[None, 6, 6, 1]], output shape: [None, 4, 4, 32]
Layer name: conv_act1, layer type: Activation, input shapes: [[None, 4, 4, 32]], output shape: [None, 4, 4, 32]
Layer name: pool, layer type: MaxPooling2D, input shapes: [[None, 4, 4, 32]], output shape: [None, 2, 2, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 2, 2, 32]], output shape: [None, 128]
Layer name: bn_dense, layer type: BatchNormalization, input

/home/bertelem/Documents/env_python3/lib/python3.7/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


Done


In [7]:
### PREDICT 
print("Evaluate normal")
results = model.evaluate(X_pca_test[:1000], y_test[:1000])
print("test acc:", results)

### PREDICT PRUNE
print("Evaluate prunning")
results = qmodel.evaluate(X_pca_test[:1000], y_test[:1000])
print("test acc:", results)

### PREDICT HLS
print("Evaluate HLS")
results_hls = hls_model_q.evaluate(X_pca_test, y_test)
print("test acc:", results)

Evaluate normal
32/32 [==============================] - 0s 3ms/step - loss: 0.2247 - accuracy: 0.9010
test acc: [0.22474217414855957, 0.9010000228881836]
Evaluate prunning


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
hls_model_q.build(csim=False, synth=True, vsynth=True)

In [ ]:
hls4ml.report.read_vivado_report('quantized_pruned_cnn/myproject_prj/solution1')